In [103]:
import json, os
import pandas as pd
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
morph = MorphAnalyzer()

In [24]:
pd.set_option('display.max_colwidth', 1000)

In [3]:
PATH_TO_DATA = './data'

In [5]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

In [6]:
files[0]

'./data/ng_0.jsonlines.zip'

In [ ]:
texts = []
keywords = []
titles = []

for file in files:
    for line in open(file):
        line = json.loads(line)
        keywords, title, text = line['keywords'], line['title'], line['content']
        

In [9]:
data = pd.read_json(files[0], lines=True)


In [25]:
data.loc[0:10, ["keywords"]]


,keywords
0,"[яблоко, молодежь, молодежное яблоко]"
1,"[газпром, газ]"
2,"[франсуа рабле, сервантес, шекспир, конан дойл, михаил булгаков, александр грин, борхес, босх, маркес, герман гессе, голландская живопись, гаргантюа и пантагрюэль, дон кихот, мастер и маргарита, москва, россия, история, поэзия, шотландия, баллада, пере]"
3,"[владивосток, суд, ким, футина, выборы, боевое братство]"
4,"[новая москва, подмосковье, благоустройство, тинао, городская среда, парки]"
5,"[автоспорт, формула1, гонки, хэмилтон, квят, сша]"
6,"[ведущие политики, рейтинг, февраль]"
7,"[законы, культура, концепция, уголовные дела, платформа, аресты]"
8,"[сирия, вооруженный конфликт, газ, углеводороды, турецкий поток]"
9,"[фсб, иг, калужская область]"


In [115]:
from string import punctuation
from nltk.corpus import stopwords
punct = set(punctuation+'«»—…“”*№–')
stops = set(stopwords.words('russian'))

def delete_punct_on_edges(word):
    if len(word) < 2:
        
        if not word or word in punct:
            return None
        else:
            # если какая-то буква то выдаем
            return word

    if word[0] not in punct:

        if word[-1] not in punct:
            return word
        else:
            return delete_punct_on_edges(word[:-1])
    else:
        return delete_punct_on_edges(word[1:])
      


def normalize(text):
    
    words = [delete_punct_on_edges(word) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and not word in stops]


    return words

In [122]:
texts = data['content'].apply(normalize).values
keywords = data['keywords'].values

In [127]:
texts[1]

['вчера',
 'газпром',
 'снизить',
 'верхний',
 'планка',
 'прогноз',
 'собственный',
 'добыча',
 'газа',
 '2020',
 'год',
 '12',
 'год',
 'концерн',
 'собираться',
 'добывать',
 'около',
 '620–640',
 'миллиард',
 'куб',
 'метр',
 'год',
 'общий',
 'производство',
 'газа',
 'страна',
 'расчёт',
 'холдинг',
 'должный',
 'достигнуть',
 '940',
 'миллиард',
 'куб',
 'метр',
 'иной',
 'слово',
 'треть',
 'добывать',
 'объём',
 'мнение',
 'холдинг',
 'должный',
 'быть',
 'обеспечить',
 'независимый',
 'производитель',
 'эксперт',
 'верить',
 'независимый',
 'компания',
 'смочь',
 'выйти',
 'такой',
 'объём',
 'добыча',
 'расчёт',
 'газпром',
 'оправдаться',
 'удар',
 'оказаться',
 'отечественный',
 'предприятие',
 'население',
 'который',
 'прийтись',
 'сокращать',
 'потребление',
 'смириться',
 'новый',
 'виток',
 'цена',
 'иной',
 'путь',
 'покрытие',
 'возможный',
 'дефицит',
 'газа',
 'вряд',
 'холдинг',
 'разорвать',
 'заключённый',
 'контракт',
 'экспорт',
 'газа',
 'другой',
 'страна',

In [44]:
from itertools import combinations, permutations

In [45]:
?combinations

In [216]:
def get_kws(text, top=5):
#     text = texts[1]

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}

    ids = [word2id[word] for word in text]

    m = np.zeros((len(vocab), len(vocab)))

    for i in range(0, len(ids), 3):
        window = ids[i:i+3]

        for j, k in combinations(window, 2):

            m[j][k] += 1
            m[k][j] += 1
    
    for i in range(m.shape[0]):
        m[i] /= np.sum(m[i])

    c = Counter()
    n = np.random.choice(len(vocab))
    for i in range(5000):
        
        j = np.random.randint(1,10)
        if j > 8:
            n = np.random.choice(len(vocab))
        n = take_step(n, m)
        c.update([n])
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    next_n = np.random.choice(range(rang), p=matrix[n])
    return next_n
    


In [220]:
Counter(texts[11]).most_common(9)

[('жильё', 15),
 ('метр', 13),
 ('кв', 12),
 ('год', 10),
 ('миллион', 10),
 ('жилищный', 9),
 ('население', 8),
 ('строительство', 6),
 ('спрос', 6)]

In [219]:
get_kws(texts[11], 9)

['жильё',
 'метр',
 'кв',
 'жилищный',
 'миллион',
 '2014-го',
 'год',
 'александр',
 'прежний']

In [221]:
keywords[11]

['многоквартирное жилье',
 'строительство',
 'президент',
 'послание',
 'минстрой',
 'прогноз',
 'доходы',
 'население']